# PBDOT Calculator

This code automatically detects the parameters in your par file.
You have two options.

Option A.

Calculate PBDOT by GW additionally by providing mass array from, for instance, TempoNest

Option B.

Just find the intrinsic PBDOT by subtracting Shklovskii effect, Galactic acceleration from the measured PBDOT.
It depends on your decision.

You can control the option by controlling "int_only" variable.

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
from astropy import units as u
from astropy.constants import G
from astropy.constants import c
from astropy import constants as const
from astropy.coordinates import SkyCoord

class params:
    
    def __init__(self):
        pass
    
    def call_parameters(name):
        call=(open(name).read())
        call=call.split('\n')

        param_array=np.zeros(0)
        for i in range(0,len(call)):
            each_line=np.array(call[i].split(' ')) 
            each_line=each_line[each_line!='']
            if each_line[0]=='CLK':
                break
            else:
                pass
            param_array=np.append(param_array,each_line)
        return param_array
    
    def param_finder(array,param):
        param_idx=np.where(array==param)
        param_name=array[param_idx]
        param_val=array[param_idx[0]+1]
        param_err=array[param_idx[0]+3]
        if param=='RAJ' or param=='DECJ': 
            if param=='RAJ':
                head='h'
                head_sub='d'
                param_idx_sub=np.where(array=='DECJ')
                param_name_sub=array[param_idx_sub]
                param_val_sub=array[param_idx_sub[0]+1]
                param_err_sub=array[param_idx_sub[0]+3]
            if param=='DECJ':
                head='d'
                head_sub='h'
                param_idx_sub=np.where(array=='RAJ')
                param_name_sub=array[param_idx_sub]
                param_val_sub=array[param_idx_sub[0]+1]
                param_err_sub=array[param_idx_sub[0]+3]

            coord=''
            err_part=''
            locator=0

            for i in param_val[0]:
                if i==':': 
                    if locator==0:
                        i=head

                    locator=locator+1
                if locator<=1:
                    coord=coord+i
                if locator>1:
                    err_part=err_part+i

            upper=float(err_part[1:])+float(param_err[0])
            lower=float(err_part[1:])-float(param_err[0])

            coord_upper=coord+'m'+str(upper)
            coord_lower=coord+'m'+str(lower)

            coord_sub=''
            err_part_sub=''
            locator_sub=0
            
            for i in param_val_sub[0]:
                if i==':': 
                    if locator_sub==0:
                        i=head_sub
                    locator_sub=locator_sub+1
                if locator_sub<=1:
                    coord_sub=coord_sub+i
                if locator_sub>1:
                    err_part_sub=err_part_sub+i


            sub_part=float(err_part_sub[1:])
            coord_sub=coord_sub+'m'+str(sub_part)
             

            if param=='RAJ':
                coord_lw= SkyCoord(coord_upper, coord_sub, frame='icrs')
                coord_up= SkyCoord(coord_lower, coord_sub, frame='icrs')  
                mean=(coord_lw.galactic.frame.l.deg+coord_up.galactic.frame.l.deg)/2
                err=(mean-coord_lw.galactic.frame.l.deg)

            if param=='DECJ':
                coord_lw= SkyCoord(coord_sub, coord_upper, frame='icrs')
                coord_up= SkyCoord(coord_sub, coord_lower, frame='icrs')            
                mean=(coord_lw.galactic.frame.b.deg+coord_up.galactic.frame.b.deg)/2
                err=(mean-coord_lw.galactic.frame.b.deg)

            param_val=mean
            param_err=err
                
        else:
            param_val=float(param_val[0])
            param_err=float(param_err[0])
      
        return param_name,param_val,param_err

class PK(params):
    def __init__(self):
        pass
    
    def pbdot(mass_m2, mass_m1, d, dd, param,iterations,model,int_only=False):
        pb=params.param_finder(array_al,'PB')[1]
        dpb=params.param_finder(array_al,'PB')[2]
        pmra=params.param_finder(array_al,'PMRA')[1]
        dpmra=params.param_finder(array_al,'PMRA')[2]
        pmdec=params.param_finder(array_al,'PMDEC')[1]
        dpmdec=params.param_finder(array_al,'PMDEC')[2]
        l=params.param_finder(array_al,'RAJ')[1]
        dl=abs(params.param_finder(array_al,'RAJ')[2])
        b=params.param_finder(array_al,'DECJ')[1]
        db=abs(params.param_finder(array_al,'DECJ')[2])
        if model=='ELL1' or model=='ELL1H':
            eps1=params.param_finder(array_al,'EPS1')[1]
            eps2=params.param_finder(array_al,'EPS2')[1]
            deps1=params.param_finder(array_al,'EPS1')[2]
            deps2=params.param_finder(array_al,'EPS2')[2]
            e1=np.random.normal(eps1,deps1,iterations)
            e2=np.random.normal(eps2,deps2,iterations)
            e=np.sqrt(e1**2+e2**2)
        else:
            e=params.param_finder(array_al,'ECC')[1]
            de=params.param_finder(array_al,'ECC')[2]
        
        def peak_finder(data):
            '''Returns the peak location in the resulting mcmc distribution.'''
            mean = np.mean(data)
            mode = 3*np.median(data)-2*np.mean(data)
            st_dev = np.std(data)
            points = mean-st_dev, mean+st_dev
            low=points[0]-mode
            up=points[1]-mode
            return mode,points[0],points[1],low,up 
        
        def pbd_gw(mass_m2, mass_m1, e,pb):
            t0=4.925490947*1e-6
            pb=np.random.normal(pb,dpb,iterations)*u.day.to('second')
            m=mass_m2+mass_m1
            return -192*np.pi/5*(pb/2/np.pi)**(-5/3)*(1+73/24*e**2+37/96*e**4)*(1-e**2)**(-7/2)*t0**(5/3)*mass_m2*mass_m1*m**(-1/3)


        def pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations):

            l=np.random.normal(l,dl,iterations)*u.deg.to('radian')#np.pi/180 #rad
            b=np.random.normal(b,db,iterations)*u.deg.to('radian') #rad
            pb=np.random.normal(pb,dpb,iterations)*u.day.to('second') #s
            
            d=np.random.normal(d,dd,iterations)*u.pc.to('meter') #m
            ro=np.random.normal(8275,34,iterations)*u.pc.to('meter') #m
            theta=np.random.normal(240500,41000,iterations) #m/s

            beta=d/ro*np.cos(b)-np.cos(l)
            z=abs(d*np.sin(b))/1000/u.pc.to('meter') #kpc
            k=(2.27*z+3.68*(1-np.exp(-4.31*z)))*1e-11 #m/s2
            k=np.random.normal(k,abs(k*0.1),iterations)
            c= const.c.value #m/s

            pbd_gal=-k*abs(np.sin(b))/c-theta**2/ro/c*(np.cos(l)+beta/(beta**2+np.sin(l)**2))*np.cos(b)
            gal=(pbd_gal*pb)
            return gal


        def pbd_shk(pmra,dpmra,pmdec,dpmdec,d,dd,pb,dpb,iterations):    


            ua=np.random.normal(pmra,dpmra,iterations)*4.8481368110954e-9 #mas/yr to rad/yr
            ud=np.random.normal(pmdec,dpmdec,iterations)*4.8481368110954e-9
            u=ua**2+ud**2
            d= np.random.normal(d,dd,iterations)*3.09e+16 # to m
            c=const.c.value/3.17098e-8 # to m/yt
            pb=np.random.normal(pb,dpb,iterations)*0.00273973 # to yr
            u=(ua**2+ud**2)
            shk=u*pb*d/c
            return shk
        
        # intrinsic pbdot based on the obtained values
        gal=np.mean(pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations))
        shk=np.mean(pbd_shk(pmra,dpmra,pmdec,dpmdec,d,dd,pb,dpb,iterations))
        gal_err=np.std(pbd_gal(d,dd,pb,dpb,l,dl,b,db,iterations))
        shk_err=np.std(pbd_shk(pmra,dpmra,pmdec,dpmdec,d,dd,pb,dpb,iterations))
        #err=np.sqrt(abs(dpbd**2-gal_err**2-shk_err**2))

        #mass-mass calculation
        if int_only==False:
            pb_gw_dist=pbd_gw(mass_m2, mass_m1,e,pb)
            pb_gw=peak_finder(pb_gw_dist)[0]
            pb_gw_up=peak_finder(pb_gw_dist)[4]
            pb_gw_low=peak_finder(pb_gw_dist)[3]

            print('gal:',gal, u"   \u00B1" ,gal_err)
            print('shk:',shk, u"   \u00B1" ,shk_err)
            print('gw:',pb_gw, "    +" ,pb_gw_up, ' ' ,pb_gw_low)
            print('--------------------------------')
            total_err_up=np.sqrt(gal_err**2+shk_err**2+pb_gw_up**2)
            total_err_low=np.sqrt(gal_err**2+shk_err**2+pb_gw_low**2)

            ext_err=np.sqrt(gal_err**2+shk_err**2)
            print('ext:', gal+shk, "    \u00B1" ,ext_err)
            print('total:', gal+shk+pb_gw, "    +" ,total_err_up, "   -" ,total_err_low)
            print('      ')
        if int_only==True:

            print('gal:',gal, u"   \u00B1" ,gal_err)
            print('shk:',shk, u"   \u00B1" ,shk_err)
            print('--------------------------------')
            ext_err=np.sqrt(gal_err**2+shk_err**2)
            print('ext:', gal+shk, "    \u00B1" ,ext_err)
            print('      ')
        #return gal,gal_err,shk,shk_err,pb_gw,pb_gw_err
        
    
# Parameters
def mass_function(params):
    A1=params.param_finder(array_al,'A1')[1]
    A1_err=params.param_finder(array_al,'A1')[2]    
    pb=params.param_finder(array_al,'PB')[1]
    pb_err=params.param_finder(array_al,'PB')[2]
    f=(4*np.pi**2/G*(np.around(A1,12)*c.value)**3/(np.around(pb,12)*u.d.to('second'))**2/u.Msun.to('kg')).value
    f_err=(4*np.pi**2/G*c.value**3/u.d.to('second')**2/u.Msun.to('kg')*np.sqrt(
        (3*A1**2*A1_err/pb**2)**2+
        (2*pb_err*A1**3/pb**3)**2)).value
    return f, f_err


# 1. call directory
directory="/path/par_file.par"
array_al=params.call_parameters(directory)


# 2. provide the measured mass and mass function.
#If you do not want to calculate PBDOT by GWB, theoretically, omit this part and use 
#PK.pbdot(.....,subtract_only=False) in the next part.
int_only=False
if int_only==False:
    mpdf=pd.read_csv('/path/mass_pdf.csv')
    mass_m1=mpdf['Mp']
    mass_m2=mpdf['Mc']
    f= mass_function(params)[0] #around 0.227596605870
    df=mass_function(params)[1] #around 0.000000027349
    binary_model='ELL1H'
if int_only==True:
    pass


# 3. iteration of the distance
d=np.linspace(600,1200,6)
error=0.3 #30% of error assumed
iterations=len(mass_m1)
for distance in d:
    print('d:', distance)
    dd=error*distance
    
    pbd=PK.pbdot(mass_m2,  mass_m1,  distance, dd, params, iterations, binary_model,int_only)
